In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
import keras
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
#config.log_device_placement = True  # to log device placement (on which device the operation ran)
                                    # (nothing gets printed in Jupyter, only if you run it standalone)
sess = tf.Session(config=config)
set_session(sess) 

/home/vle/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
from __future__ import print_function

import keras
from keras.models import Sequential
from keras.layers import Conv1D, Dense, Dropout, Flatten, BatchNormalization
from keras.callbacks import EarlyStopping, CSVLogger

from loader import load_20news
from custom_layer import MonteCarloLRF, SeparableMonteCarloLRF, SeparableMonteCarloMaxPoolingV2, RandomLRF

import numpy as np
import sklearn as sk
import pickle

In [4]:
top_words=10000
sparse=False
remove_short_documents=True
notebook = 'mcNet_top10k_temptative_42'

In [5]:
(input_shape, nb_classes), (X_train, X_test, Y_train, Y_test), graph_data = \
    load_20news(data_home='data', top_words=top_words, sparse=sparse,
                remove_short_documents=remove_short_documents, verbose=False)

In [6]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2])
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2])
num_classes = Y_train.shape[1]

In [7]:
%%time
#Process next cell only once
path = os.path.join('probabilities_' + 
                    'top' + str(top_words) +
                    '_sparse' + str(sparse) +
                    '_removeShorts' + str(remove_short_documents) +
                    '.pkl')
if os.path.isfile(path):
  probabilities = pickle.load(open(path, "rb"), encoding='latin1')

CPU times: user 2.18 s, sys: 1.17 s, total: 3.35 s
Wall time: 3.35 s


In [8]:
if not(os.path.isfile(path)):
  METRIC = 'cosine'#'euclidean'
  distances = sk.metrics.pairwise.pairwise_distances(graph_data, metric=METRIC, n_jobs=-2)

  # enforce exact zero
  for k in xrange(distances.shape[0]):
    distances[k,k] = 0.

  # max normalize
  #distances /= distances.max()
  distances /= distances.max(axis=1).reshape((distances.shape[0], 1))

  # use tricube kernel (becaause of flatness around 0)
  probabilities = (1. - np.abs(distances) ** 3) ** 3

  # remove auto connections (which are taken anyway in LRF)
  for k in xrange(probabilities.shape[0]):
    probabilities[k,k] = 0.

  # normalize proba
  probabilities /= np.sum(probabilities, axis=1).reshape((probabilities.shape[0], 1))
  
  # pickled for later use
  pickle.dump(probabilities, open(path,"wb"))  

In [9]:
batch_size = 64
model = Sequential()
model.add(RandomLRF(probabilities, LRF_size=2, filters=1, activation='relu',
                       input_shape=(X_train.shape[1], X_train.shape[2])))
#model.add(BatchNormalization(axis=1))
#model.add(Conv1D(1, kernel_size=1, activation='relu',
#                 padding='same',kernel_initializer='he_uniform'))
#model.add(SeparableMonteCarloMaxPoolingV2(LRF_size=4, new_size=2500))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
random_lrf_1 (RandomLRF)     (None, 10000, 1)          3         
_________________________________________________________________
dropout_1 (Dropout)          (None, 10000, 1)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 10000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 500)               5000500   
_________________________________________________________________
dropout_2 (Dropout)          (None, 500)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                10020     
Total params: 5,010,523
Trainable params: 5,010,523
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

early_stopper = EarlyStopping(min_delta=0.001, patience=2)
csv = CSVLogger(notebook + '_log.csv')

history = model.fit(X_train, Y_train,
                    batch_size=batch_size,
                    epochs=100,
                    verbose=1,
                    callbacks=[early_stopper, csv],
                    validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 10168 samples, validate on 7071 samples
Epoch 1/100
10168/10168 [==============================] - 3s 268us/step - loss: 2.7645 - acc: 0.3586 - val_loss: 2.3464 - val_acc: 0.4964
Epoch 2/100
10168/10168 [==============================] - 2s 213us/step - loss: 1.6840 - acc: 0.6769 - val_loss: 1.4860 - val_acc: 0.6464
Epoch 3/100
10168/10168 [==============================] - 2s 213us/step - loss: 0.9505 - acc: 0.7982 - val_loss: 1.2290 - val_acc: 0.6694
Epoch 4/100
10168/10168 [==============================] - 2s 213us/step - loss: 0.6189 - acc: 0.8672 - val_loss: 1.1410 - val_acc: 0.6867
Epoch 5/100
10168/10168 [==============================] - 2s 213us/step - loss: 0.4323 - acc: 0.9055 - val_loss: 1.1342 - val_acc: 0.6883
Epoch 6/100
10168/10168 [==============================] - 2s 214us/step - loss: 0.3033 - acc: 0.9375 - val_loss: 1.1559 - val_acc: 0.6869
Epoch 7/100
10168/10168 [==============================] - 2s 213us/step - loss: 0.2201 - acc: 0.9577 - val_loss: 1.2

In [11]:
%%bash
tail mcNet_top10k_temptative_42_log.csv

epoch,acc,loss,val_acc,val_loss
0,0.35857592442202624,2.76446945642881,0.49639372076412996,2.3464097147900995
1,0.6769276159096664,1.6840244514748959,0.6464432188230834,1.4860172715454791
2,0.7981904013526431,0.9504622590063502,0.6693536982460788,1.2290241229552694
3,0.8672305272377728,0.6188702034162405,0.6867486917471856,1.1409636864718142
4,0.9054878047373612,0.43234659092431926,0.68830434157612,1.1342023759764233
5,0.9374508260273727,0.3032921240800577,0.6868901145861149,1.1558690719667881
6,0.9577104642952079,0.220057494088386,0.6873143827320073,1.2061432455877759
